In [267]:
# import pandas
import pandas as pd
import re
import nltk.corpus
nltk.download('stopwords')
# import stopwords
# import TfidfVectorizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
stopworda = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# import data
df = pd.read_csv("https://raw.githubusercontent.com/nikitaa30/Content-based-Recommender-System/master/sample-data.csv")

In [44]:
df[df['id']==11]

,id,description,desc_lower
10,11,"Baby sunshade top - Soft, stretchy polyester f...","baby sunshade top - soft, stretchy polyester f..."


* explore DataFrame

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   description  500 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [9]:
df['description']

0      Active classic boxers - There's a reason why o...
1      Active sport boxer briefs - Skinning up Glory ...
2      Active sport briefs - These superbreathable no...
3      Alpine guide pants - Skin in, climb ice, switc...
4      Alpine wind jkt - On high ridges, steep ice an...
                             ...                        
495    Cap 2 bottoms - Cut loose from the maddening c...
496    Cap 2 crew - This crew takes the edge off fick...
497    All-time shell - No need to use that morning T...
498    All-wear cargo shorts - All-Wear Cargo Shorts ...
499    All-wear shorts - Time to simplify? Our All-We...
Name: description, Length: 500, dtype: object

In [10]:
df['desc_lower'] = df['description'].str.lower()

In [11]:
df.head()

,id,description,desc_lower
0,1,Active classic boxers - There's a reason why o...,active classic boxers - there's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...,active sport boxer briefs - skinning up glory ...
2,3,Active sport briefs - These superbreathable no...,active sport briefs - these superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc...","alpine guide pants - skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an...","alpine wind jkt - on high ridges, steep ice an..."


In [268]:
clean_spl_ch = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z 3+_]') 
def text_cleaning(text):
    text = clean_spl_ch.sub(' ',text)
    text = clean_symbol.sub('',text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

In [269]:
df['desc_lower'] = df['desc_lower'].apply(text_cleaning)
df.head()

,id,description,desc_lower
0,1,Active classic boxers - There's a reason why o...,active classic boxers theres reason boxers cul...
1,2,Active sport boxer briefs - Skinning up Glory ...,active sport boxer briefs skinning glory requi...
2,3,Active sport briefs - These superbreathable no...,active sport briefs superbreathable nofly brie...
3,4,"Alpine guide pants - Skin in, climb ice, switc...",alpine guide pants skin climb ice switch rock ...
4,5,"Alpine wind jkt - On high ridges, steep ice an...",alpine wind jkt high ridges steep ice anything...


We will be using Tf-Idf to find similar items based on description
* instantiate TF-IDF

In [270]:
tf = TfidfVectorizer(analyzer='word',min_df=0,stop_words='english')

* fit and transform 'description' column with TFIDF

In [271]:
tf_matrix = tf.fit_transform(df['desc_lower'])

In [272]:
print(tf_matrix)

  (0, 3679)	0.10190876382351943
  (0, 1017)	0.025901993032341537
  (0, 3998)	0.025901993032341537
  (0, 219)	0.11028287639816789
  (0, 796)	0.1559577919032029
  (0, 4261)	0.03448957098502915
  (0, 2268)	0.09191496189802881
  (0, 1418)	0.08929813754550832
  (0, 3902)	0.08323949362378458
  (0, 3812)	0.08581660102870667
  (0, 2300)	0.09191496189802881
  (0, 535)	0.06300706194045785
  (0, 722)	0.10635332034051964
  (0, 5662)	0.025491655710190367
  (0, 3184)	0.07257877782619464
  (0, 2943)	0.16629376884159133
  (0, 1340)	0.05910919222089834
  (0, 1835)	0.07459232321262407
  (0, 3072)	0.1559577919032029
  (0, 2910)	0.10196662284076147
  (0, 4338)	0.13435876656223136
  (0, 1022)	0.06345429544175181
  (0, 5670)	0.11668929727890809
  (0, 1973)	0.04290797836788639
  (0, 1128)	0.08749797883499899
  :	:
  (499, 4458)	0.1646524183670849
  (499, 4819)	0.16805767671909858
  (499, 6005)	0.10359576433295602
  (499, 3213)	0.0878800844866681
  (499, 5479)	0.09494104465795818
  (499, 4180)	0.2259182334201

In [273]:
print(tf.get_feature_names_out())
#print(tf.get_stop_words())
print(tf.get_params())

['000' '06' '07' ... 'zipsecurity' 'zipthrough' 'zones']
{'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.float64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 0, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': 'english', 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


* calculate the cosine similarity of each item with every other item in the dataset, 

In [254]:
csm = cosine_similarity(tf_matrix,tf_matrix) 
csm

array([[1.        , 0.2223454 , 0.0877353 , ..., 0.08836737, 0.08784452,
        0.07926607],
       [0.2223454 , 1.        , 0.51724951, ..., 0.06491962, 0.10999822,
        0.06398209],
       [0.0877353 , 0.51724951, 1.        , ..., 0.05106509, 0.04763025,
        0.0438856 ],
       ...,
       [0.08836737, 0.06491962, 0.05106509, ..., 1.        , 0.04608666,
        0.05669134],
       [0.08784452, 0.10999822, 0.04763025, ..., 0.04608666, 1.        ,
        0.53933123],
       [0.07926607, 0.06398209, 0.0438856 , ..., 0.05669134, 0.53933123,
        1.        ]])

In [255]:
csm.shape

(500, 500)

* sort all items using their similarity for each item i, and store the values in dictionary `results`

```
results = {
    "1": [5,7,9...],
    "2": [45,2,3...]
}
```

In [256]:
name = pd.Series(df.index)

In [257]:
results={}
for i in range(len(df['id'])):
    item = pd.Series(csm[i]).sort_values(ascending=False).index ,
    results[i] =list(item)
    

In [258]:
results

{0: [Int64Index([  0,  18, 493,  17, 494, 364, 170, 441, 171, 495,
              ...
               16,  48, 240, 243, 252,   9, 136, 133, 421, 433],
             dtype='int64', length=500)],
 1: [Int64Index([  1,   2, 299,  18,   0, 493, 298,  17, 155, 494,
              ...
              343, 237,  68, 318, 395, 489, 331,  47, 141, 370],
             dtype='int64', length=500)],
 2: [Int64Index([  2,   1, 494, 299, 298, 155, 154,  18, 257, 493,
              ...
              237,  68, 318,  85, 395, 489, 331,  47, 370, 141],
             dtype='int64', length=500)],
 3: [Int64Index([  3, 158, 383, 342, 378, 352, 212, 215, 114, 119,
              ...
              433, 147,   7, 219, 344, 237,  68, 395, 489,  47],
             dtype='int64', length=500)],
 4: [Int64Index([  4, 307,  95, 280, 203, 183, 437, 215, 415, 292,
              ...
              243, 252, 240, 136, 237, 395,  47,  68, 489, 331],
             dtype='int64', length=500)],
 5: [Int64Index([  5, 437, 183, 414, 267

* create function `recommender` that will recommend similar products
    * function must have two input params: **item_id** and **count** of similar products 

In [259]:
def recommendations(item_id, csm=csm):
    recommendations=[]
    idx = name[name==item_id].index[0]
    score_series = pd.Series(csm[idx]).sort_values(ascending=False)
    top_5_indexes = list(score_series.iloc[1:7].index)
    for i in top_5_indexes:
        recommendations.append(list(df.index)[i])
    
    return recommendations

In [260]:
saved_recomm = recommendations(11)
saved_recomm

[401, 427, 407, 465, 311, 324]

* show top 5 the most similar items for item with idem_id = 11

In [141]:
for i in saved_recomm:
    desc = (df[df['id']==i])
    print(desc['description'])

400    Reversible phone home - Travel broadly and sur...
Name: description, dtype: object
406    Merino 2 crew - In this soft long-sleeved tee,...
Name: description, dtype: object
426    Gi ii zip-off pants - The multitalented Gi II ...
Name: description, dtype: object
310    Astrid wrap - A casual, elegant wrap to layer ...
Name: description, dtype: object
464    Baby baggies apron dress - This lively dress k...
Name: description, dtype: object
323    Vitalti pants - Soft and stretchy Vitaliti pan...
Name: description, dtype: object
